## Cohere hackathon

- Build a multimodal capability abstraction
- Build an example based on that

In [1]:
from dotenv import load_dotenv
from rich import print

load_dotenv("../.env")

import os
os.environ["LOGURU_LEVEL"] = "DEBUG"

In [2]:
from agents.tools import wikipedia_search, get_current_weather, image_inspector

from agents.llms._cohere import CohereChatCompletion
from agents.tool_executor import need_tool_use

In [3]:
llm = CohereChatCompletion()

In [4]:
llm.bind_tools([get_current_weather, wikipedia_search, image_inspector])

In [10]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that has access to tools. Use the tools only if you don't have answers."
    },
    {"role":"user", "content":"how is the weather in London today?"}
]

output = llm.chat_completion(messages)

if need_tool_use(output):
    tool_results = llm.run_tool(output)
    tool_results[0]["role"] = "assistant"

    updated_messages = messages + tool_results
    updated_messages = updated_messages + [{"role":"system", "content": "Do not use any tools now."},
                                           {"role": "user", "content": "Think step by step and answer my question based on the above context."}
                                          ]
    print(updated_messages)
    output = llm.chat_completion(updated_messages)

print(output.choices[0].message.content)

2024-05-08 16:49:50.210 | DEBUG    | agents.llms._cohere:chat_completion:59 - content='' additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'fa720e92-d178-4845-b9f6-e415a6177bc9', 'tool_calls': [{'id': '65ae4caf67c94d01a1f8b5f88a306996', 'function': {'name': 'get_current_weather', 'arguments': '{"city": "London"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 10}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'fa720e92-d178-4845-b9f6-e415a6177bc9', 'tool_calls': [{'id': '65ae4caf67c94d01a1f8b5f88a306996', 'function': {'name': 'get_current_weather', 'arguments': '{"city": "London"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 10}} id='run-2acf6171-f2a9-4f5c-a589-90a23ad7c9e8-0' tool_calls=[{'name': 'get_current_weather', 'args': {'city': 'London'}, 'id': 'e6cbeb6a

[
    {
        'role': 'system',
        'content': "You are a helpful assistant that has access to tools. Use the tools only if you don't have 
answers."
    },
    {'role': 'user', 'content': 'how is the weather in London today?'},
    {
        'tool_call_id': '65ae4caf67c94d01a1f8b5f88a306996',
        'role': 'assistant',
        'name': 'get_current_weather',
        'content': '{"FeelsLikeC": "20", "FeelsLikeF": "68", "cloudcover": "50", "humidity": "49", 
"localObsDateTime": "2024-05-08 02:58 PM", "observation_time": "01:58 PM", "precipInches": "0.0", "precipMM": 
"0.0", "pressure": "1029", "pressureInches": "30", "temp_C": "20", "temp_F": "68", "uvIndex": "5", "visibility": 
"10", "visibilityMiles": "6", "weatherCode": "116", "weatherDesc": [{"value": "Partly cloudy"}], "weatherIconUrl": 
[{"value": ""}], "winddir16Point": "SE", "winddirDegree": "140", "windspeedKmph": "7", "windspeedMiles": "4"}'
    },
    {'role': 'system', 'content': 'Do not use any tools now.'},
    {'role': 'user', 'content': 'Think step by step and answer my question based on the above context.'}
]

2024-05-08 16:49:54.720 | DEBUG    | agents.llms._cohere:chat_completion:59 - content="The weather in London, as of 1:58 PM, is partly cloudy with a temperature of 20 degrees Celsius or 68 degrees Fahrenheit. The humidity is at 49%, and the UV index is 5. There is no precipitation, and the wind speed is mild at 7 km/h or 4 mph. The pressure is at 1029 millibars, with a visibility range of 10 km or 6 miles. So, overall, it's a pleasant day with mild temperatures and partly cloudy skies." additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '0d276416-263e-4a18-b037-5aed320e473f', 'token_count': {'input_tokens': 349, 'output_tokens': 115}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '0d276416-263e-4a18-b037-5aed320e473f', 'token_count': {'input_tokens': 349, 'output_tokens': 115}} id='run-

The weather in London, as of 1:58 PM, is partly cloudy with a temperature of 20 degrees Celsius or 68 degrees 
Fahrenheit. The humidity is at 49%, and the UV index is 5. There is no precipitation, and the wind speed is mild at
7 km/h or 4 mph. The pressure is at 1029 millibars, with a visibility range of 10 km or 6 miles. So, overall, it's 
a pleasant day with mild temperatures and partly cloudy skies.

In [ ]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

messages = [
    {"role": "system", "content": "You are a helpful assistant that has access to tools and use that to help humans."},
    {"role": "user", "content": f"Check this image {image_url} and suggest me a location where I can go in London which looks similar"}
]
output = llm.chat_completion(messages)

if need_tool_use(output):
    tool_results = llm.run_tool(output)
    tool_results[0]["role"] = "assistant"

    updated_messages = messages + tool_results
    updated_messages = updated_messages + [{"role":"system", "content": "Do not use any tools now."},
                                           {"role": "user", "content": "Think step by step and answer my question based on the above context."}
                                          ]
    print(updated_messages)
    output = llm.chat_completion(updated_messages)

print(output.choices[0].message.content)

2024-05-08 16:50:28.194 | DEBUG    | agents.llms._cohere:chat_completion:59 - content='' additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '8112dfdc-8f1c-420c-a381-b6a132073f4a', 'tool_calls': [{'id': 'f213c3e072644e9cb24795ad3df81549', 'function': {'name': 'image_inspector', 'arguments': '{"image_url_or_path": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 73}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '8112dfdc-8f1c-420c-a381-b6a132073f4a', 'tool_calls': [{'id': 'f213c3e072644e9cb24795ad3df81549', 'function': {'name': 'image_inspector', 'arguments': '{"image_url_or_path": "https://upload.wikimedia.org/wikipedi

[
    {
        'role': 'system',
        'content': 'You are a helpful assistant that has access to tools and use that to help humans.'
    },
    {
        'role': 'user',
        'content': 'Check this image 
https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp
-wisconsin-madison-the-nature-boardwalk.jpg and suggest me a location where I can go in London which looks similar'
    },
    {
        'tool_call_id': 'f213c3e072644e9cb24795ad3df81549',
        'role': 'assistant',
        'name': 'image_inspector',
        'content': " The image depicts a long wooden boardwalk stretching across a lush green field. The boardwalk 
is surrounded by tall grass and wildflowers, creating a picturesque scene of nature's beauty. In the distance, 
there are trees visible, adding to the serene atmosphere of the landscape."
    },
    {'role': 'system', 'content': 'Do not use any tools now.'},
    {'role': 'user', 'content': 'Think step by step and answer my question based on the above context.'}
]